<a href="https://colab.research.google.com/github/mistryvivek/YRKCS-PRBX/blob/main/Model_1a_Basic_NN_(Earth_movers_distance).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


# Setup Weights and Bias.

* AI experimental tool.

In [3]:
!pip install wandb -qU

In [4]:
# Log in to your W&B account
import wandb
import random
import math

In [5]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Earth Movers Distance Loss

* https://github.com/TakaraResearch/Pytorch-1D-Wasserstein-Statistical-Loss/blob/master/pytorch_stats_loss.py

In [6]:
import torch
from torch.autograd import Variable


#######################################################
#       STATISTICAL DISTANCES(LOSSES) IN PYTORCH      #
#######################################################

## Statistial Distances for 1D weight distributions
## Inspired by Scipy.Stats Statistial Distances for 1D
## Pytorch Version, supporting Autograd to make a valid Loss
## Supposing Inputs are Groups of Same-Length Weight Vectors
## Instead of (Points, Weight), full-length Weight Vectors are taken as Inputs
## Code Written by E.Bao, CASIA

def torch_wasserstein_loss(tensor_a,tensor_b):
    #Compute the first Wasserstein distance between two 1D distributions.
    return(torch_cdf_loss(tensor_a,tensor_b,p=1))

def torch_energy_loss(tensor_a,tensor_b):
    # Compute the energy distance between two 1D distributions.
    return((2**0.5)*torch_cdf_loss(tensor_a,tensor_b,p=2))

def torch_cdf_loss(tensor_a,tensor_b,p=1):
    # last-dimension is weight distribution
    # p is the norm of the distance, p=1 --> First Wasserstein Distance
    # to get a positive weight with our normalized distribution
    # we recommend combining this loss with other difference-based losses like L1

    # normalize distribution, add 1e-14 to divisor to avoid 0/0
    tensor_a = tensor_a / (torch.sum(tensor_a, dim=-1, keepdim=True) + 1e-14)
    tensor_b = tensor_b / (torch.sum(tensor_b, dim=-1, keepdim=True) + 1e-14)
    # make cdf with cumsum
    cdf_tensor_a = torch.cumsum(tensor_a,dim=-1)
    cdf_tensor_b = torch.cumsum(tensor_b,dim=-1)

    # choose different formulas for different norm situations
    if p == 1:
        cdf_distance = torch.sum(torch.abs((cdf_tensor_a-cdf_tensor_b)),dim=-1)
    elif p == 2:
        cdf_distance = torch.sqrt(torch.sum(torch.pow((cdf_tensor_a-cdf_tensor_b),2),dim=-1))
    else:
        cdf_distance = torch.pow(torch.sum(torch.pow(torch.abs(cdf_tensor_a-cdf_tensor_b),p),dim=-1),1/p)

    cdf_loss = cdf_distance.mean()
    return cdf_loss

def torch_validate_distibution(tensor_a,tensor_b):
    # Zero sized dimension is not supported by pytorch, we suppose there is no empty inputs
    # Weights should be non-negetive, and with a positive and finite sum
    # We suppose all conditions will be corrected by network training
    # We only check the match of the size here
    if tensor_a.size() != tensor_b.size():
        raise ValueError("Input weight tensors must be of the same size")

## Modify code for sliding window approach (like wk11 code).

* Look at the full race image instead.
* What data types are in the wk11 practical.
* No vocab size required because they apply to catogorical contexts.

In [7]:
BASE_PATH = r"/content/drive/MyDrive/prbx_data/v1/"
max_race_size = 0
max_tyre_life = 0

# Compound Encoder

In [23]:
compound_encoder = LabelEncoder()
all_compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
compound_encoder.fit(all_compounds)

LabelEncoder()

In [32]:
# TRAINING
training_inputs = []
training_outputs = []

RaceCalender22 = pd.read_csv(BASE_PATH + r"2022/eventCalender2022.csv")
for _, row in RaceCalender22.iterrows():
    if row['EventFormat'] != 'testing':
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2022/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values  # Add the compound data

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            stint_change_array = [0 if stint_array[i] == stint_array[i + 1] else 1
                                  for i in range(len(stint_array) - 1)] + [0]
            stint_change_array = np.array(stint_change_array)

            encoded_compounds = compound_encoder.transform(compound_array)

            mandatory_pit_stop_made = []
            # Make mandatory pit made list.
            for compound in encoded_compounds:
              if len(mandatory_pit_stop_made) > 0 and mandatory_pit_stop_made[-1] == 1:
                mandatory_pit_stop_made.append(1)
              elif compound != encoded_compounds[0]:
                mandatory_pit_stop_made.append(1)
              elif compound in [compound_encoder.transform(['INTERMEDIATE'])[0], compound_encoder.transform(['WET'])[0]]:
                mandatory_pit_stop_made.append(1)
              else:
                mandatory_pit_stop_made.append(0)

            # Combine tyre_life and compound as input features
            combined_input = np.column_stack((tyre_life_array, encoded_compounds, mandatory_pit_stop_made))  # Combine into a 2D array
            training_inputs.append(combined_input)
            training_outputs.append(stint_change_array)

In [33]:
# TESTING
testing_inputs = []
testing_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
for _, row in RaceCalender23.iterrows():
    if row['EventFormat'] != 'testing':
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2023/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values  # Add the compound data

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            stint_change_array = [0 if stint_array[i] == stint_array[i + 1] else 1
                                  for i in range(len(stint_array) - 1)] + [0]
            stint_change_array = np.array(stint_change_array)

            encoded_compounds = compound_encoder.transform(compound_array)

            mandatory_pit_stop_made = []
            # Make mandatory pit made list.
            for compound in encoded_compounds:
              if len(mandatory_pit_stop_made) > 0 and mandatory_pit_stop_made[-1] == 1:
                mandatory_pit_stop_made.append(1)
              elif compound != encoded_compounds[0]:
                mandatory_pit_stop_made.append(1)
              elif compound in [compound_encoder.transform(['INTERMEDIATE'])[0], compound_encoder.transform(['WET'])[0]]:
                mandatory_pit_stop_made.append(1)
              else:
                mandatory_pit_stop_made.append(0)

            # Combine tyre_life and compound as input features
            combined_input = np.column_stack((tyre_life_array, encoded_compounds, mandatory_pit_stop_made))  # Combine into a 2D array
            testing_inputs.append(combined_input)
            testing_outputs.append(stint_change_array)

In [34]:
class RNNv1bDropout(nn.Module):
    def __init__(self,input_size,hidden_size,dropout_prob): #vocab_size
        super(RNNv1bDropout, self).__init__()
        #self.embedding =  nn.Embedding(vocab_size, hidden_size)
        self.linear_x = nn.Linear(input_size, hidden_size)
        self.linear_h = nn.Linear(hidden_size,hidden_size)
        self.linear_y = nn.Linear(hidden_size,1)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self,x,hprev):
        """
        h = self.tanh(self.embedding(x) + self.linear_h(hprev))
        y = self.linear_y(h)
        """
        h = self.tanh(self.linear_x(x) + self.linear_h(hprev))
        h - self.dropout(h)
        y = self.sigmoid(self.linear_y(h))
        return h,y

In [35]:
def calculate_loss(model,inputs,targets,hprev):
  loss_func = torch_wasserstein_loss
  seq_length = len(inputs)
  outputs = []
  for t in range(seq_length):
    # For each character in the input sequence, pass through RNN with previous hidden state
    hprev,y = model(inputs[t],hprev)
    # Gradually build up matrix of output logits of size seq_length * vocab_size - we want it at every time step do not want this.
    # Compute cross entropy loss for seq_length actual targets against estimated distributions
    # RuntimeError: result type Float can't be cast to the desired output type Long
    outputs.append(y)

  outputs = torch.stack(outputs).squeeze(1)
  loss = loss_func(targets, outputs)

  # For truncated backprop, the next subsequence will use the final hidden state
  # but will not backprop through it so we need to detach
  hprev = hprev.detach()

  return loss, hprev

In [47]:
# Model parameters to be logged by w and b.
hidden_size = 6
lr = 0.002
iterations = 10000
input_parameters = ['TyreLife', 'Compound']
dataset = 'v1'
input_size = 3
dropout = 0.45

In [48]:
# PADDING
def add_padding(max_race_size, sequence):
    if len(sequence) >= max_race_size:
        return sequence[:max_race_size]
    else:
        # Handle both 1D and 2D arrays
        if sequence.ndim == 1:  # If 1D array
            padding_shape = (max_race_size - len(sequence),)
        else:  # If 2D array
            padding_shape = (max_race_size - len(sequence), sequence.shape[1])

        padding = np.zeros(padding_shape, dtype=sequence.dtype)
        return np.concatenate([sequence, padding])

In [49]:
def check_accuracy(model):
  total, matches = 0, 0
  no_pit_prediction_total, correct_no_pit_prediction = 0,0
  pit_prediction_total, correct_pit_prediction = 0,0

  for t in range(len(testing_inputs) - 1):
      inputs = add_padding(int(max_race_size),testing_inputs[t])
      outputs = add_padding(int(max_race_size),testing_outputs[t])
      hprev = torch.zeros(hidden_size, device=device)

      for x in range(len(inputs) - 1):

        hprev, y = model(torch.tensor([inputs[x]], device=device, dtype=torch.float32), hprev)

        if (y.item() > 0.5):
          pit_prediction_total += 1
          if 1 in outputs[max(0, x - 2): min(int(max_race_size), x+4)]:
              correct_pit_prediction += 1
        else:
          no_pit_prediction_total += 1
          if y.floor().item() == outputs[x]:
              correct_no_pit_prediction += 1

  if (pit_prediction_total + no_pit_prediction_total) != 0:
      overall_accuracy = (correct_no_pit_prediction + correct_pit_prediction) / (no_pit_prediction_total + pit_prediction_total) * 100
      print(f"Overall Accuracy: {overall_accuracy:.2f}%")
      wandb.log({"overall_accuracy": overall_accuracy})
  else:
      print("Overall Accuracy: 0.0%")
      wandb.log({"overall_accuracy": 0.0})

  if no_pit_prediction_total != 0:
    print(f"Total No Pit Accuracy: {correct_no_pit_prediction / no_pit_prediction_total * 100}")
    wandb.log({"total no pit accurary (exact match)": correct_no_pit_prediction / no_pit_prediction_total * 100})
  else:
    print(f"Total No Pit Accuracy: 0.0")
    wandb.log({"total no pit accurary (exact match)": 0.0})

  if pit_prediction_total != 0:
    print(f"Total Pit Accuracy: {correct_pit_prediction / pit_prediction_total * 100}")
    wandb.log({"total pit accurary (3 laps either side)": correct_pit_prediction / pit_prediction_total * 100})
  else:
    print(f"Total Pit Accuracy: 0.0")
    wandb.log({r"total pit accurary (3 laps either side)": 0.0})

  print(f"Total Pit Predictions: {pit_prediction_total}")
  wandb.log({"total pit predictions": pit_prediction_total})

In [50]:
def train(training_inputs, training_outputs):
  model = RNNv1bDropout(input_size,hidden_size,dropout)#vocab_size
  model.to(device)

  optim = torch.optim.Adagrad(model.parameters(), lr = lr)

  config = {
      "learning_rate": lr,
      "hidden_size": hidden_size,
      "iterations": 10000,
      "input_parameters": input_parameters,
      "dataset": dataset,
      "input_size": input_size,
      "loss_function": "torch_wasserstein_loss",
      "dropout": dropout
  }

  wandb.init(project="binary_classifier_emd", config=config)

  # Shuffle the data before starting the training loop
  training_data = list(zip(training_inputs, training_outputs))
  random.shuffle(training_data)
  training_inputs, training_outputs = zip(*training_data)

  n, p = 0, 0
  while n<=iterations:
    hprev = torch.zeros(hidden_size, device=device) # reset RNN memory

    training_data = list(zip(training_inputs, training_outputs))
    random.shuffle(training_data)
    training_inputs, training_outputs = zip(*training_data)

    # Why apply padding here
    inputs = add_padding(int(max_race_size),training_inputs[p])
    outputs = add_padding(int(max_race_size),training_outputs[p])

    # Extract next subsequence of characters
    inputs = torch.tensor(training_inputs[p],dtype=torch.float32, device=device)
    targets = torch.tensor(training_outputs[p],dtype=torch.float32, device=device)

    # Compute loss for current subsequence
    loss, hprev = calculate_loss(model,inputs,targets,hprev)
    optim.zero_grad()
    loss.backward()
    optim.step()

    wandb.log({"loss": loss.item()})

    if n % 500 == 0:
      print('[{:}] Loss: {:.2f}'.format(n,loss.item()))
      check_accuracy(model)

    p += 1 # move data pointer
    n += 1 # iteration counter

    if p >= len(training_inputs):  # Check if we've gone through all data
      # Shuffle the data before starting a new epoch
      training_data = list(zip(training_inputs, training_outputs))
      random.shuffle(training_data)
      training_inputs, training_outputs = zip(*training_data)

      p = 0

  # Optionally, save the entire PyTorch model object
  torch.save(model, "full_model.pth")
  wandb.save("full_model.pth")

#train(training_inputs, training_outputs)

In [54]:
wandb.finish()

In [52]:
# Define the sweep configuration
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 1e-4,
            'max': 1e-2
        },
        'optimizer': {
            'values': ['adam', 'sgd', 'rmsprop', 'adagrad']
        },
        'hidden_size': {
            'min': 1,
            'max': 50
        },
        'dropout': {
            'min': 0.1,
            'max': 0.5
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="binary_classifier_emd")

Create sweep with ID: 0sg2xel9
Sweep URL: https://wandb.ai/vm800-university-of-york/binary_classifier_emd/sweeps/0sg2xel9


In [53]:
# Run the sweep
wandb.agent(sweep_id, function=lambda: train(training_inputs, training_outputs), count=10)

wandb: Agent Starting Run: fahav4be with config:
wandb: 	dropout: 0.2708183050772066
wandb: 	hidden_size: 21
wandb: 	learning_rate: 0.00021407340575059963
wandb: 	optimizer: sgd
wandb: WARNING Ignoring project 'binary_classifier_emd' when running a sweep.


[0] Loss: 6.57


<ipython-input-49-e7617c931dd8>:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  hprev, y = model(torch.tensor([inputs[x]], device=device, dtype=torch.float32), hprev)


Overall Accuracy: 73.44%
Total No Pit Accuracy: 96.4117390787754
Total Pit Accuracy: 4.392857142857142
Total Pit Predictions: 8400
[500] Loss: 8.05
Overall Accuracy: 73.06%
Total No Pit Accuracy: 96.40670319046085
Total Pit Accuracy: 7.388101983002833
Total Pit Predictions: 8825
[1000] Loss: 7.04
Overall Accuracy: 74.08%
Total No Pit Accuracy: 96.46038816417436
Total Pit Accuracy: 7.901234567901234
Total Pit Predictions: 8505
[1500] Loss: 1.00
Overall Accuracy: 74.14%
Total No Pit Accuracy: 96.4608104346443
Total Pit Accuracy: 8.139261350270525
Total Pit Predictions: 8502
[2000] Loss: 16.55
Overall Accuracy: 75.81%
Total No Pit Accuracy: 96.53683022685708
Total Pit Accuracy: 8.79245283018868
Total Pit Predictions: 7950
[2500] Loss: 8.95
Overall Accuracy: 90.92%
Total No Pit Accuracy: 97.10587929240374
Total Pit Accuracy: 25.267518122195376
Total Pit Predictions: 2897
[3000] Loss: 8.42
Overall Accuracy: 92.08%
Total No Pit Accuracy: 97.14074597616218
Total Pit Accuracy: 29.5796986518636

loss,▅▂▅▃▃▇▂▅▆▂▃▄▃█▅▅▄▁▇▂▄▂▂▃█▄▃▅▅▂▄▅▇▃▃▃▅▃▄▆
overall_accuracy,▁▁▁▁▂▇███████████████
total no pit accurary (exact match),▁▁▁▂▂▇███████████████
total pit accurary (3 laps either side),▁▂▂▂▂▆▇▇▇▇▇█████████▇
total pit predictions,████▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
loss,6.15506
overall_accuracy,91.37567
total no pit accurary (exact match),97.12017
total pit accurary (3 laps either side),30.87731
total pit predictions,2918


...

[0] Loss: 17.64
Overall Accuracy: 15.37%
Total No Pit Accuracy: 88.93805309734513
Total Pit Accuracy: 14.371780582582764
Total Pit Predictions: 33197
[500] Loss: 17.45
Overall Accuracy: 68.68%
Total No Pit Accuracy: 96.1294304355493
Total Pit Accuracy: 11.892384860921112
Total Pit Predictions: 10965
[1000] Loss: 7.38
Overall Accuracy: 68.89%
Total No Pit Accuracy: 96.14354109017438
Total Pit Accuracy: 11.882006984010292
Total Pit Predictions: 10882
[1500] Loss: 7.65
Overall Accuracy: 68.43%
Total No Pit Accuracy: 96.11723557798733
Total Pit Accuracy: 11.887542939793889
Total Pit Predictions: 11062
[2000] Loss: 13.83
Overall Accuracy: 69.21%
Total No Pit Accuracy: 96.16546290216678
Total Pit Accuracy: 12.19918548685672
Total Pit Predictions: 10804
[2500] Loss: 13.59
Overall Accuracy: 91.70%
Total No Pit Accuracy: 97.10302193285324
Total Pit Accuracy: 40.98208770846201
Total Pit Predictions: 3238
[3000] Loss: 16.39
Overall Accuracy: 91.76%
Total No Pit Accuracy: 97.10007208860345
Total P

loss,▇▆▆█▄▃▃▆▅▇▅▄▆▅▃▅█▅▅▆▃▂▃▃▇▃▁▃▃▆▂▃▃▃▅▅▄▆▃▅
overall_accuracy,▁▆▆▆▆████████████████
total no pit accurary (exact match),▁▇▇▇▇████████████████
total pit accurary (3 laps either side),▁▁▁▁▁▇▆▆▇▇███████████
total pit predictions,█▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,11.05194
overall_accuracy,93.02802
total no pit accurary (exact match),97.14981
total pit accurary (3 laps either side),47.77501
total pit predictions,2809


wandb: Agent Starting Run: dir2hlc5 with config:
wandb: 	dropout: 0.4930507039199285
wandb: 	hidden_size: 5
wandb: 	learning_rate: 0.001299051845845068
wandb: 	optimizer: sgd
wandb: WARNING Ignoring project 'binary_classifier_emd' when running a sweep.


[0] Loss: 8.87
Overall Accuracy: 40.98%
Total No Pit Accuracy: 99.7886305484481
Total Pit Accuracy: 19.53771289537713
Total Pit Predictions: 24660
[500] Loss: 13.99
Overall Accuracy: 50.99%
Total No Pit Accuracy: 98.48185820555639
Total Pit Accuracy: 20.434676434676437
Total Pit Predictions: 20475
[1000] Loss: 15.87
Overall Accuracy: 58.06%
Total No Pit Accuracy: 97.61079700893671
Total Pit Accuracy: 20.22674418604651
Total Pit Predictions: 17200
[1500] Loss: 13.04
Overall Accuracy: 62.23%
Total No Pit Accuracy: 97.36798995249276
Total Pit Accuracy: 20.272561293688053
Total Pit Predictions: 15336
[2000] Loss: 6.06
Overall Accuracy: 65.20%
Total No Pit Accuracy: 97.22477298234874
Total Pit Accuracy: 20.516836335160534
Total Pit Predictions: 14047
[2500] Loss: 9.61
Overall Accuracy: 67.61%
Total No Pit Accuracy: 97.17865294031952
Total Pit Accuracy: 20.971200980392158
Total Pit Predictions: 13056
[3000] Loss: 9.71
Overall Accuracy: 68.36%
Total No Pit Accuracy: 97.1733307824756
Total Pit

loss,▆▂▃▄▆▄▅▄▁▆▃▅▅▆█▅▆▅▄▃▃▄▂▄▃▅▄▄▃▅▃▄▄▃▄▅▃▃▄▃
overall_accuracy,▁▃▄▅▆▆▆▆▇▇▇▇▇▇███████
total no pit accurary (exact match),█▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total pit accurary (3 laps either side),▁▂▂▂▂▃▃▄▄▄▅▅▅▅▆▇▇▇▇▇█
total pit predictions,█▆▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
loss,13.39761
overall_accuracy,78.28464
total no pit accurary (exact match),97.05623
total pit accurary (3 laps either side),24.57831
total pit predictions,8715


## Lets try a LTSM varient of a RNN.

In [55]:
def check_accuracy_ltsm(model):
    total, matches = 0, 0
    no_pit_prediction_total, correct_no_pit_prediction = 0, 0
    pit_prediction_total, correct_pit_prediction = 0, 0

    # Loop through the testing data
    for t in range(len(testing_inputs) - 1):
        inputs = add_padding(int(max_race_size), testing_inputs[t])
        outputs = add_padding(int(max_race_size), testing_outputs[t])

        # Convert the entire sequence to tensor and add batch dimension
        input_tensor = torch.tensor([inputs], device=device, dtype=torch.float32)
        target_tensor = torch.tensor([outputs], device=device, dtype=torch.float32)

        # Run the model on the full input sequence
        y, (hprev, cprev) = model(input_tensor)

        # Remove the extra batch dimension
        y = y.squeeze(0)

        # Process the predictions and compare with actual outputs
        for x in range(len(y)):
            # For pit predictions (y > 0.5)
            if y[x].item() > 0.5:
                pit_prediction_total += 1
                if 1 in outputs[max(0, x - 2): min(int(max_race_size), x + 4)]:
                    correct_pit_prediction += 1
            else:
                no_pit_prediction_total += 1
                if y[x].floor().item() == outputs[x]:
                    correct_no_pit_prediction += 1

    # Calculate overall accuracy
    if (pit_prediction_total + no_pit_prediction_total) != 0:
        overall_accuracy = (correct_no_pit_prediction + correct_pit_prediction) / (no_pit_prediction_total + pit_prediction_total) * 100
        print(f"Overall Accuracy: {overall_accuracy:.2f}%")
        wandb.log({"overall_accuracy": overall_accuracy})
    else:
        print("Overall Accuracy: 0.0%")
        wandb.log({"overall_accuracy": 0.0})

    # Calculate accuracy for no pit predictions
    if no_pit_prediction_total != 0:
        print(f"Total No Pit Accuracy: {correct_no_pit_prediction / no_pit_prediction_total * 100:.2f}%")
        wandb.log({"total no pit accuracy (exact match)": correct_no_pit_prediction / no_pit_prediction_total * 100})
    else:
        print(f"Total No Pit Accuracy: 0.0")
        wandb.log({"total no pit accuracy (exact match)": 0.0})

    # Calculate accuracy for pit predictions
    if pit_prediction_total != 0:
        print(f"Total Pit Accuracy: {correct_pit_prediction / pit_prediction_total * 100:.2f}%")
        wandb.log({"total pit accuracy (3 laps either side)": correct_pit_prediction / pit_prediction_total * 100})
    else:
        print(f"Total Pit Accuracy: 0.0")
        wandb.log({"total pit accuracy (3 laps either side)": 0.0})

    print(f"Total Pit Predictions: {pit_prediction_total}")
    wandb.log({"total pit predictions": pit_prediction_total})

In [56]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_prob):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    # https://discuss.pytorch.org/t/lstm-model-not-learning/110164
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_dim, device=device)
        c0 = torch.zeros(1, x.size(0), self.hidden_dim, device=device)
        out, (h0, c0) = self.lstm(x, (h0, c0))  # h0, c0 are the hidden and cell states
        y = self.sigmoid(self.linear(out))  # Use the last time step's output
        return y, (h0, c0)

In [57]:
def calculate_loss_ltsm(model, inputs, targets):
    loss_func = torch_wasserstein_loss
    outputs, hprev = model(inputs)
    outputs = outputs.squeeze(-1)

    loss = loss_func(targets, outputs)

    hprev = tuple(state.detach() for state in hprev)

    return loss, hprev

In [58]:
def train(training_inputs, training_outputs):
    model = LSTMModel(input_size, hidden_size, dropout).to(device)
    optim = torch.optim.Adagrad(model.parameters(), lr=lr)

    # Log configuration to W&B
    config = {
        "learning_rate": lr,
        "hidden_size": hidden_size,
        "iterations": iterations,
        "input_parameters": input_parameters,
        "dataset": dataset,
        "input_size": input_size,
        "loss_function": "torch_wasserstein_loss",
        "dropout": dropout,
    }

    wandb.init(project="binary_classifier_emd_lstm", config=config)

    # Pad all sequences to the same length
    max_length = max(len(seq) for seq in training_inputs)
    training_inputs = [add_padding(max_length, seq) for seq in training_inputs]
    training_outputs = [add_padding(max_length, seq) for seq in training_outputs]

    # Convert inputs and outputs to tensors
    training_inputs = torch.tensor(training_inputs, dtype=torch.float32, device=device)
    training_outputs = torch.tensor(training_outputs, dtype=torch.float32, device=device)

    n = 0  # iteration counter
    while n <= iterations:
        # Shuffle the data at the start of each iteration
        data = list(zip(training_inputs, training_outputs))
        random.shuffle(data)
        training_inputs, training_outputs = zip(*data)

        # Iterate through the dataset one sequence at a time
        for i in range(len(training_inputs)):
            inputs = training_inputs[i].unsqueeze(0)  # Add batch dimension
            targets = training_outputs[i].unsqueeze(0)  # Add batch dimension

            # Compute loss
            loss, (hprev, cprev) = calculate_loss_ltsm(model, inputs, targets)

            # Backpropagation
            optim.zero_grad()
            loss.backward()
            optim.step()

            # Log metrics
            wandb.log({"loss": loss.item()})

            # Print progress every 500 iterations
            if n % 500 == 0:
                print(f'[Iteration {n}] Loss: {loss.item():.2f}')
                check_accuracy_ltsm(model)

            n += 1
            if n > iterations:
                break

    # Save model
    torch.save(model, "full_model.pth")
    wandb.save("full_model.pth")

In [59]:
# Define the sweep configuration
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 1e-4,
            'max': 1e-2
        },
        'optimizer': {
            'values': ['adam', 'sgd', 'rmsprop', 'adagrad']
        },
        'hidden_size': {
            'min': 1,
            'max': 50
        },
        'dropout': {
            'min': 0.1,
            'max': 0.5
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="binary_classifier_emd_lstm")

Create sweep with ID: v7ifde9r
Sweep URL: https://wandb.ai/vm800-university-of-york/binary_classifier_emd_lstm/sweeps/v7ifde9r


In [60]:
# Run the sweep
wandb.agent(sweep_id, function=lambda: train(training_inputs, training_outputs), count=10)

wandb: Agent Starting Run: 75uybrw0 with config:
wandb: 	dropout: 0.382032909016666
wandb: 	hidden_size: 26
wandb: 	learning_rate: 0.0089343572935654
wandb: 	optimizer: adagrad
wandb: WARNING Ignoring project 'binary_classifier_emd_lstm' when running a sweep.


[Iteration 0] Loss: 23.53
Overall Accuracy: 97.33%
Total No Pit Accuracy: 97.33%
Total Pit Accuracy: 0.0
Total Pit Predictions: 0
[Iteration 500] Loss: 9.04
Overall Accuracy: 97.22%
Total No Pit Accuracy: 97.33%
Total Pit Accuracy: 25.45%
Total Pit Predictions: 55
[Iteration 1000] Loss: 11.54
Overall Accuracy: 65.67%
Total No Pit Accuracy: 97.13%
Total Pit Accuracy: 18.20%
Total Pit Predictions: 13586
[Iteration 1500] Loss: 17.93
Overall Accuracy: 55.89%
Total No Pit Accuracy: 97.89%
Total Pit Accuracy: 20.39%
Total Pit Predictions: 18470
[Iteration 2000] Loss: 17.43
Overall Accuracy: 52.07%
Total No Pit Accuracy: 98.17%
Total Pit Accuracy: 20.31%
Total Pit Predictions: 20181
[Iteration 2500] Loss: 14.01
Overall Accuracy: 50.42%
Total No Pit Accuracy: 98.39%
Total Pit Accuracy: 20.34%
Total Pit Predictions: 20949
[Iteration 3000] Loss: 17.92
Overall Accuracy: 50.19%
Total No Pit Accuracy: 98.46%
Total Pit Accuracy: 20.43%
Total Pit Predictions: 21088
[Iteration 3500] Loss: 10.76
Overal

loss,▆▄▂▄▂▂█▃▃▄▂▂▂▃▃▅▂▂▃▃▃▁▅▁▂▃▃▁▄▃▃▄▃▃▂▂▁▄▃▃
overall_accuracy,██▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total no pit accuracy (exact match),▂▂▁▄▆▇▇▇▇▇▇███▇▇▇▇▇▇▆
total pit accuracy (3 laps either side),▁█▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
total pit predictions,▁▁▅▇█████████████████
loss,11.75936
overall_accuracy,52.40275
total no pit accuracy (exact match),98.35244
total pit accuracy (3 laps either side),20.53066
total pit predictions,20126


wandb: Agent Starting Run: oncg7oed with config:
wandb: 	dropout: 0.1023608680741543
wandb: 	hidden_size: 44
wandb: 	learning_rate: 0.00974892895419182
wandb: 	optimizer: adam
wandb: WARNING Ignoring project 'binary_classifier_emd_lstm' when running a sweep.


[Iteration 0] Loss: 17.89
Overall Accuracy: 27.45%
Total No Pit Accuracy: 98.90%
Total Pit Accuracy: 13.57%
Total Pit Predictions: 28544
[Iteration 500] Loss: 21.83
Overall Accuracy: 14.31%
Total No Pit Accuracy: 0.0
Total Pit Accuracy: 14.31%
Total Pit Predictions: 34086
[Iteration 1000] Loss: 22.07
Overall Accuracy: 34.23%
Total No Pit Accuracy: 100.00%
Total Pit Accuracy: 17.87%
Total Pit Predictions: 27294
[Iteration 1500] Loss: 7.01
Overall Accuracy: 36.86%
Total No Pit Accuracy: 99.94%
Total Pit Accuracy: 18.43%
Total Pit Predictions: 26377
[Iteration 2000] Loss: 17.24
Overall Accuracy: 37.71%
Total No Pit Accuracy: 99.88%
Total Pit Accuracy: 18.55%
Total Pit Predictions: 26056
[Iteration 2500] Loss: 7.09
Overall Accuracy: 38.66%
Total No Pit Accuracy: 99.75%
Total Pit Accuracy: 18.67%
Total Pit Predictions: 25680
[Iteration 3000] Loss: 16.46
Overall Accuracy: 39.17%
Total No Pit Accuracy: 99.74%
Total Pit Accuracy: 18.75%
Total Pit Predictions: 25489
[Iteration 3500] Loss: 13.37

...

[Iteration 0] Loss: 7.67
Overall Accuracy: 14.47%
Total No Pit Accuracy: 100.00%
Total Pit Accuracy: 14.30%
Total Pit Predictions: 34019
[Iteration 500] Loss: 17.37
Overall Accuracy: 38.19%
Total No Pit Accuracy: 100.00%
Total Pit Accuracy: 18.80%
Total Pit Predictions: 25944
[Iteration 1000] Loss: 15.78
Overall Accuracy: 39.15%
Total No Pit Accuracy: 99.96%
Total Pit Accuracy: 19.02%
Total Pit Predictions: 25611
[Iteration 1500] Loss: 14.11
Overall Accuracy: 39.59%
Total No Pit Accuracy: 99.94%
Total Pit Accuracy: 19.13%
Total Pit Predictions: 25457
[Iteration 2000] Loss: 11.44
Overall Accuracy: 40.05%
Total No Pit Accuracy: 99.92%
Total Pit Accuracy: 19.21%
Total Pit Predictions: 25285
[Iteration 2500] Loss: 12.74
Overall Accuracy: 40.83%
Total No Pit Accuracy: 99.80%
Total Pit Accuracy: 19.32%
Total Pit Predictions: 24973
[Iteration 3000] Loss: 38.89
Overall Accuracy: 41.45%
Total No Pit Accuracy: 99.81%
Total Pit Accuracy: 19.47%
Total Pit Predictions: 24762
[Iteration 3500] Loss: 

loss,▆▂█▂▄█▇▄▅▃▂▂▂▂█▁▁▃▂▂▅▅▄▆▄▂▄▅▃▆▂▂▃▃▄▂▄▄▂▄
overall_accuracy,▁▅▅▅▆▆▆▆▆▇▇▇▇▇▇██████
total no pit accuracy (exact match),█████▇▇▇▅▅▄▄▃▃▃▂▂▂▁▁▁
total pit accuracy (3 laps either side),▁▆▆▆▆▆▆▆▇▇▇▇▇▇███████
total pit predictions,█▄▄▄▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁
loss,8.5582
overall_accuracy,54.16593
total no pit accuracy (exact match),98.42828
total pit accuracy (3 laps either side),21.12113
total pit predictions,19516


wandb: Agent Starting Run: cmk1ig6l with config:
wandb: 	dropout: 0.4238988791538242
wandb: 	hidden_size: 13
wandb: 	learning_rate: 0.0017505169497908145
wandb: 	optimizer: rmsprop
wandb: WARNING Ignoring project 'binary_classifier_emd_lstm' when running a sweep.


[Iteration 0] Loss: 18.92
Overall Accuracy: 64.93%
Total No Pit Accuracy: 96.47%
Total Pit Accuracy: 12.90%
Total Pit Predictions: 12862
[Iteration 500] Loss: 10.17
Overall Accuracy: 51.83%
Total No Pit Accuracy: 98.37%
Total Pit Accuracy: 20.24%
Total Pit Predictions: 20302
[Iteration 1000] Loss: 7.85
Overall Accuracy: 48.32%
Total No Pit Accuracy: 98.82%
Total Pit Accuracy: 20.15%
Total Pit Predictions: 21879
[Iteration 1500] Loss: 5.80
Overall Accuracy: 47.30%
Total No Pit Accuracy: 99.07%
Total Pit Accuracy: 20.17%
Total Pit Predictions: 22367
[Iteration 2000] Loss: 17.50
Overall Accuracy: 48.07%
Total No Pit Accuracy: 99.10%
Total Pit Accuracy: 20.40%
Total Pit Predictions: 22101
[Iteration 2500] Loss: 7.14
Overall Accuracy: 48.68%
Total No Pit Accuracy: 99.07%
Total Pit Accuracy: 20.53%
Total Pit Predictions: 21868
[Iteration 3000] Loss: 10.37
Overall Accuracy: 49.50%
Total No Pit Accuracy: 98.97%
Total Pit Accuracy: 20.65%
Total Pit Predictions: 21529
[Iteration 3500] Loss: 6.02

loss,▃▄▂▃▁▁▄▂▁▂▂▇▂▂▁▇▂▂▂██▂▂▂▂▂▂▁▂▁▂▃▃▁▃▃▂▃▂▂
overall_accuracy,█▃▁▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇
total no pit accuracy (exact match),▁▆▇████▇▇▇▇▇▆▆▆▆▆▆▆▅▅
total pit accuracy (3 laps either side),▁▇▇▇▇▇▇▇▇▇███████████
total pit predictions,▁▆████▇▇▇▆▆▆▅▅▅▄▄▄▄▄▃
loss,15.15219
overall_accuracy,62.26603
total no pit accuracy (exact match),98.10935
total pit accuracy (3 laps either side),21.73533
total pit predictions,15997
